In this notebook, different statistics of the detected ramp planes will be analyzed and visualized.<br>
Also a score will be calculated to determine how well the algorithm works.<br>
This will be done for multiple rosbags to get an average score.

In [ ]:
import numpy as np
import pandas as pd
import pcl
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import ros_numpy
import rosbag
from tf.transformations import (
    euler_from_quaternion,
    euler_matrix,
    unit_vector,
    vector_norm,
    quaternion_matrix,
)

pio.templates.default = "plotly_dark"
# %load_ext blackcellmagic

#### **The following values need to be adjusted for each rosbag**

In [ ]:
# Rosbags path
bag_directory = '/home/user/rosbags/final/slam/'

# ROS topics
odom_topic = '/odom'
lidar_topic = '/velodyne_points'

Helper functions

In [ ]:
def unpack_bag(bag_path, topic):
    """Extracts all msgs (with time) of a topic from a rosbag"""
    # Load rosbag
    bag = rosbag.Bag(bag_path)
    
    t_msg = []
    msgs = []
    for topic, msg, t in bag.read_messages(topics=topic):
        if topic == odom_topic:
            msgs.append(msg.pose.pose)
        else:
            msgs.append(msg)
        # Time at which msg was recorded
        t_msg.append(t.to_time())
    return msgs, t_msg


def synchronize_topics(topic1, t_topic1, topic2, t_topic2, t_thresh=0.1):
    """Synchronizes two topics (even if different rate)."""
    if len(topic1) == len(topic2):
        return topic1, topic2

    # Make sure both topics start around the same time and remove first values of
    # signal which is ahead if this is not the case
    topic1, t_topic1, topic2, t_topic2 = correct_for_time_diff(
        topic1, t_topic1, topic2, t_topic2)

    # Assume first array is bigger
    array_big = t_topic1
    array_small = t_topic2
    is_topic1_bigger = True
    # Swap if not
    if len(t_topic1) < len(t_topic2):
        array_small, array_big = array_big, array_small
        is_topic1_bigger = False
    indices = []

    # For each time of the small array, check which is
    # the closest msg in the big array
    for i, v in enumerate(array_small):
        # Offset big array with a msg from small array
        diff = np.abs(array_big - v)
        # Index where time difference is the smallest
        idx = diff.argmin()
        if diff[idx] < t_thresh:
            indices.append(idx)

    print(
        "t_start_diff: {:.3f} s, t_end_diff: {:.3f} s".format(
            np.abs(array_small[0] - array_big[indices][0]),
            np.abs(array_big[indices][-1] - array_small[len(array_big[indices]) - 1]),
        )
    )

    if is_topic1_bigger:
        topic1_synched = topic1[indices]
        topic2_synched = topic2[: len(topic1_synched)]
        return topic1_synched, topic2_synched
    else:
        topic2_synched = topic2[indices]
        topic1_synched = topic1[: len(topic2_synched)]
        return topic1_synched, topic2_synched


def correct_for_time_diff(topic1, t_topic1, topic2, t_topic2, t_thresh=0.1):
    """Checks for a time offset of two topics at the beginning and shorts ahead signal"""
    # Assume first topic is behind and convert to numpy array
    topic_behind, topic_ahead = np.asarray(topic1), np.asarray(topic2)
    t_topic_behind, t_topic_ahead = np.asarray(t_topic1), np.asarray(t_topic2)
    is_topic1_behind = True
    # Swap if other way around
    if t_topic2[0] - t_topic1[0] > t_thresh:
        topic_ahead, topic_behind = topic_behind, topic_ahead
        t_topic_ahead, t_topic_behind = t_topic_behind, t_topic_ahead
        is_topic1_behind = False
    # No offset
    elif np.abs(t_topic1[0] - t_topic2[0]) < t_thresh:
        return tuple([np.asarray(x) for x in [topic1, t_topic1, topic2, t_topic2]])

    # Search for time of ahead topic which is closest to start of behind topic
    diff = np.abs(t_topic_ahead - t_topic_behind[0])
    # Index where time difference is the smallest
    idx = diff.argmin()
    # Remove first couple values to make both start at the same time
    topic_ahead = topic_ahead[idx:]
    t_topic_ahead = t_topic_ahead[idx:]

    if is_topic1_behind:
        print("topic1 was behind by {}".format(idx))
        return topic_behind, t_topic_behind, topic_ahead, t_topic_ahead
    else:
        print("topic2 was behind by {}".format(idx))
        return topic_ahead, t_topic_ahead, topic_behind, t_topic_behind

### Ramp detection algorithm ROS node

In [ ]:
# My ROS Node
class VisualDetection():
    
    def __init__(self):
        self.ramp_data = [[] for i in range(5)]
        self.arr = np.zeros((1,3))
        # x coordinates where ramp starts
        self.ramp_start = x_range[0]

    def spin(self, cloud, pose):
        self.cloud = cloud
        self.pose = pose
        # Convert PointCloud2 msg to numpy array
        pc_array = ros_numpy.point_cloud2.pointcloud2_to_xyz_array(self.cloud, remove_nans=True)

        # Apply lidar to car frame transformation
        pc_array_tf = self.transform_pc(pc_array, rpy=(0, 0.0083, 0),
                                        translation_xyz=(-2.36, -0.05, 1.8753))

        # Reduce point cloud size with a passthrough filter
        pc_array_cut = self.reduce_pc(pc_array_tf, (0, 30), (-2, 2), (-1, 2))
            
        # Convert numpy array to pcl object
        pc_cut = self.array_to_pcl(pc_array_cut)

        # Downsample point cloud using voxel filter to further decrease size
        pc_small = self.voxel_filter(pc_cut, 0.1)

        # Perform RANSAC until no new planes are being detected
        plane_coor, data = self.plane_detection(pc_small, 20, 4)
        return plane_coor, data, self.relative_to_absolute(pc_array_cut)

    @staticmethod
    def array_to_pcl(pc_array):
        """Get pcl point cloud from numpy array"""
        pc = pcl.PointCloud()
        pc.from_array(pc_array.astype('float32'))
        return pc

    @staticmethod
    def quat_from_vectors(vec1, vec2):
        """Gets quaternion to align vector 1 with vector 2"""
        # Make sure both vectors are unit vectors
        v1_uv, v2_uv = unit_vector(vec1), unit_vector(vec2)
        cross_prod = np.cross(v1_uv, v2_uv)
        dot_prod = np.dot(v1_uv, v2_uv)

        # Rotation axis
        axis = cross_prod / vector_norm(cross_prod)
        # Rotation angle (rad)
        ang = np.arctan2(vector_norm(cross_prod), dot_prod)

        # Quaternion ([x,y,z,w])
        quat = np.append(axis*np.sin(ang/2), np.cos(ang/2))
        return quat

    @staticmethod
    def transform_pc(pc, rpy=(0, 0, 0), translation_xyz=(1.7, 0, 1.7)):
        """Transformation from lidar frame to car frame.
        Rotation in rad and translation in m."""
        # Extract euler angles
        roll, pitch, yaw = rpy
        # Extract translations
        transl_x, transl_y, transl_z = translation_xyz

        # Rotation matrix
        rot = euler_matrix(roll, pitch, yaw, 'sxyz')[:3, :3]
        # Apply rotation
        pc_tf = np.inner(pc, rot)

        # Translation to front of the car
        translation_to_front = [transl_x, transl_y, transl_z]
        # Combine rotation and translation
        pc_tf += translation_to_front
        return pc_tf

    @staticmethod
    def reduce_pc(pc, x_range, y_range, z_range):
        """Removes points outside of box"""
        # Filter array
        pc_cut = pc[
            (pc[:, 0] > x_range[0])
            & (pc[:, 0] < x_range[1])
            & (pc[:, 1] > y_range[0])
            & (pc[:, 1] < y_range[1])
            & (pc[:, 2] > z_range[0])
            & (pc[:, 2] < z_range[1])
            ]
        return pc_cut

    @staticmethod
    def voxel_filter(pc, leaf_size):
        """Downsample point cloud using voxel filter"""
        vgf = pc.make_voxel_grid_filter()
        # Leaf_size is the length of the side of the voxel cube in m
        vgf.set_leaf_size(leaf_size, leaf_size, leaf_size)
        pc_filtered = vgf.filter()
        return pc_filtered

    def plane_detection(self, pc, min_points, max_planes):
        """Detects all planes in point cloud"""
        # Count number of iterations
        counter = 0
        # Standard values for ramp angle and distance if no detection
        ramp_stats = ([], [])
        # Detect planes until ramp found or conditions not met anymore
        while pc.size > min_points and counter < max_planes:
            # Detect most dominate plane and get inliers and normal vector
            inliers_idx, coefficients = self.ransac(pc)
            # Normal vector of plane
            n_vec = coefficients[:-1]

            # Split pointcloud in outliers of plane and inliers
            plane = pc.extract(inliers_idx)
            pc = pc.extract(inliers_idx, negative=True)

            # Exit if plane is empty (RANSAC did not find anything)
            if plane.size == 0:
                return ramp_stats

            # Ignore planes parallel to the side or front walls
            if self.is_plane_near_ground(n_vec):
                # Check if ramp conditions are fullfilled
                is_ramp, data = self.ramp_detection(
                    plane, n_vec, (3, 9), (2, 6))
                # Ramp conditions met
                if is_ramp:
                    plane_global = self.relative_to_absolute(plane)
                    return (plane_global, data)
            counter += 1
        return ramp_stats
    
    def relative_to_absolute(self, pc):
        """Transforms relative lidar data to absolute by adding translation rotating"""
        # pc_arr = pc.to_array()
        pc_arr = np.array(pc)
    
        # Odometer
        pos = self.pose.position
        translation = [pos.x, pos.y, pos.z]
        ori = self.pose.orientation
        quat = [ori.x, ori.y, ori.z, ori.w]
        roll, pitch, yaw = euler_from_quaternion(quat)

        # Rotation matrix
        rot = quaternion_matrix(quat)[:3, :3]
        # Apply rotation
        pc_tf = np.inner(pc_arr, rot)

        # Combine rotation and translation
        pc_tf += translation
        return pc_tf

    def ramp_detection(self, plane, n_vec, angle_range, width_range):
        """Checks if conditions to be considered a ramp are fullfilled."""
        # Convert pcl plane to numpy array
        plane_array = plane.to_array()

        # Calculate angle [deg] between normal vector of plane and ground
        angle = self.angle_calc([0, 0, 1], n_vec)
        # Get ramp width (difference between y-values)
        width = max(plane_array[:, 1]) - min(plane_array[:, 1])
        # Ramp distance (average x-value of nearest points of the plane)
        n_nearest = 10
        dist = np.mean(np.sort(plane_array[:n_nearest, 0]))
        
        # True distance to ramp using odometer from hdl_slam
        true_dist = self.ramp_start - self.pose.position.x

        # Assert ramp angle and width thresholds
        if (angle_range[0] <= angle <= angle_range[1]
                and width_range[0] <= width <= width_range[1]):
            return True, [angle, width, dist, true_dist] 
        return False, [angle, width, dist, true_dist] 

    @staticmethod
    def ransac(pc):
        """Find inliers and normal vector of dominant plane"""
        # 50?
        seg = pc.make_segmenter_normals(50)
        # Doubles the speed if True
        seg.set_optimize_coefficients(True)
        seg.set_model_type(pcl.SACMODEL_NORMAL_PLANE)
        seg.set_method_type(pcl.SAC_RANSAC)
        # How close a point must be to model to be considered inlier
        seg.set_distance_threshold(0.11)
        # normal_distance_weight?
        seg.set_normal_distance_weight(0.01)
        # How many tries
        seg.set_max_iterations(100)
        inliers_idx, coefficients = seg.segment()
        return inliers_idx, coefficients

    @staticmethod
    def is_plane_near_ground(v, threshold=0.8):
        """Returns True if plane is on the ground (and false if e.g. side wall)"""
        # z-axis points up
        return abs(v[2]) > threshold

    @staticmethod
    def angle_calc(v1, v2, degrees=True):
        """Calculate angle between two vectors (planes)"""
        # Assuming both vectors can be rotated alongside one axis to be aligned
        dot = np.dot(v1, v2)

        # Make sure arccos is defined (dot=[0,1]) (should always be the case because
        # v1 and v2 are unit vectors, but probably due to rounding errors not always true)
        if dot <= 1:
            angle = np.arccos(dot)
        else:
            angle = 0

        if degrees is True:
            return np.degrees(angle)
        return angle

### Evaluate output/performance of detection algorithm

In [ ]:
class GetScore:
    def __init__(self, bag_name, x_range, y_range):
        self.bag_name = bag_name
        self.x_range = x_range
        self.y_range = y_range

    def extract_data(self, bag_path):
        """Get odometer and lidar data from rosbag"""
        # Odometer from hdl_slam
        pose, t_pose = unpack_bag(bag_path, odom_topic)
        # Velodyne point cloud
        lidar, t_lidar = unpack_bag(bag_path, lidar_topic)

        # Synchronize both topics (odom usually is behind lidar)
        pose, lidar = synchronize_topics(pose, t_pose, lidar, t_lidar)
        return lidar, pose

    def run_the_algorithm(self, lidar, pose):
        """Perform ramp detection on data"""
        # Create instance of class (using standard parameters):
        vd = VisualDetection()
        # Lists to fill, will contain entry for each plane
        planes = []
        ramp_stats = []
        true_dist = []
        all_points = []
        for i in range(len(lidar)):
            plane_points, data, pc_whole = vd.spin(lidar[i], pose[i])
            planes.append(plane_points)
            ramp_stats.append(data)
            all_points.append(pc_whole)
            # True distance to start of ramp
            true_dist.append(self.x_range[0] - pose[i].position.x)
        true_dist = np.array(true_dist)
        return planes, ramp_stats, true_dist

    def convert_to_df(self, planes, ramp_stats):
        """Add pointcloud to df each time a ramp has been detected"""
        # Remove empty lists (when no ramp has been detected)
        # because information such as angle, width of ramp etc
        # are only stored when a ramp is detected, 0 otherwise
        ramp_arrays = [x for x in planes if not isinstance(x, list)]
        ramp_stats = [x for x in ramp_stats if x != []]
        # Get indices where ramp has been detected
        ramp_indices = [i for i, v in enumerate(planes) if not isinstance(v, list)]

        # Convert list to dictionary
        dic = []
        print(type(ramp_arrays))
        for i, arr in enumerate(ramp_arrays):
            for j, point in enumerate(arr):
                dic.append(
                    {
                        "sampleIdx": i,
                        "pointIdx": j,
                        "x": point[0],
                        "y": point[1],
                        "z": point[2],
                    }
                )
        # And finally to pandas data frame
        df = pd.DataFrame(dic)
        # Reorder columns
        df = df[["sampleIdx", "pointIdx", "x", "y", "z"]]
        return df, ramp_stats

    def ground_truth_check(self, df, ramp_stats):
        """Check if ?"""
        # Check if a point lies within ramp region
        lies_inside = []
        for i, x in enumerate(df["x"]):
            if self.x_range[0] < x < self.x_range[1]:
                if self.y_range[0] < df["y"][i] < self.y_range[1]:
                    # True if x and y coordinate inside region
                    lies_inside.append(True)
                else:
                    lies_inside.append(False)
            else:
                lies_inside.append(False)
        # Add column (bool: if point lies in region) to data frame
        df["inlier"] = lies_inside

        # Calculate how many points of each sample lie in ramp region
        true_inliers = []
        samples_num = df.sampleIdx.max() + 1
        for i in range(samples_num):
            # Bool list of inliers and outliers of sample
            bool_lst = df[df["sampleIdx"] == i]["inlier"]
            # Percentage of inliers of sample
            true_inliers.append(sum(bool_lst) / float(len(bool_lst)))

        # New dataframe with stats for each frame
        # Structure reminder of ramp_stats: [angle, width, dist, true_dist]
        dic = []
        for i in range(samples_num):
            dic.append(
                {
                    "sampleIdx": i,
                    "TrueInliers": true_inliers[i],
                    "Angle": ramp_stats[i][0],
                    "Width": ramp_stats[i][1],
                    "Dist": ramp_stats[i][2],
                    "TrueDist": ramp_stats[i][3],
                }
            )
        # Convert dictionary to dataframe
        df_stats = pd.DataFrame(dic)
        # Reorder columns
        df_stats = df_stats[
            ["sampleIdx", "TrueInliers", "Angle", "Width", "Dist", "TrueDist"]
        ]
        return df_stats

    def some_plot(self, true_dist, df_stats):
        """Bar plot that shows detection success rate per m interval before ramp"""
        expected_detections = []
        actual_detections = []
        # Split dataset in 1m intervals
        for i in range(15, -5, -1):
            # Filter distance to ramp (from odom data) in 1m intervals
            expected_detections.append(
                len(true_dist[(i < true_dist) & (true_dist < i + 1)])
            )
            # Get corresponding distance estimations
            actual_detections.append(
                len(
                    df_stats["TrueDist"][
                        (i < df_stats["TrueDist"]) & (df_stats["TrueDist"] < i + 1)
                    ]
                )
            )
        # Create pandas data frame
        eval_df = pd.DataFrame()
        # Distance to ramp e.g. a value of 15 means interval from 16m to 15m
        eval_df["distToRamp"] = range(15, -5, -1)
        eval_df["expectedDetections"] = expected_detections
        eval_df["actualDetections"] = actual_detections

        # Plot
        fig = go.Figure()
        fig.add_trace(
            go.Bar(
                y=eval_df["expectedDetections"],
                x=eval_df["distToRamp"],
                name="False negatives (Not detected)",
                offsetgroup=0,
            )
        )
        fig.add_trace(
            go.Bar(
                y=eval_df["actualDetections"],
                x=eval_df["distToRamp"],
                name="True positives (Detected)",
                offsetgroup=0,
            )
        )
        fig.update_xaxes(title_text="Distance to ramp [m]", autorange="reversed")
        fig.update_yaxes(title_text="Number of ramps detected")
        fig.update_layout(
            title_text="How many samples were collected in 1m intervals and how many of them have been identified as ramp"
        )
        fig.show()
        return fig

    def calc_score(self, df_stats, true_dist, min_dist, max_dist):
        """Calculates some scores to determine performance"""
        # Sensitivity
        # How many sample were recorded in given range
        expected_detections = len(
            true_dist[(min_dist < true_dist) & (true_dist < max_dist)]
        )
        # How many samples were identified as ramp in given range
        actual_detections = len(
            df_stats["TrueDist"][
                (min_dist < df_stats["TrueDist"]) & (df_stats["TrueDist"] < max_dist)
            ]
        )
        # Calculate ratio
        sensitivity = float(actual_detections) / expected_detections
        print(self.bag_name)
        print(
            "Out of {} recorded samples {} were detected in the range of {} to {}m before the ramp.".format(
                expected_detections, actual_detections, min_dist, max_dist
            )
        )
        print("Resulting in a score of {:.2f}%\n".format(sensitivity * 100))

        # Inlier score
        f = df_stats["TrueInliers"][
            (min_dist < df_stats["TrueDist"]) & (df_stats["TrueDist"] < max_dist)
        ]
        score2 = sum(f) / len(f)
        print(
            "Of the {} detected planes {:.2f}% inlier points were actually inside the ramp region.\n".format(
                actual_detections, score2 * 100
            )
        )

        # False positives?
        f = df_stats["TrueInliers"][
            (min_dist < df_stats["TrueDist"]) & (df_stats["TrueDist"] < max_dist)
        ]
        thresh = 0.5
        true_detections = sum(f > thresh)
        score3 = float(true_detections) / len(f)
        print(
            "Of the {} detected planes {} had at least {}% of points inside the ramp region.".format(
                actual_detections, true_detections, int(thresh * 100)
            )
        )
        print("Resulting in a score of {:.2f}%\n\n".format(score3 * 100))

        scores = [sensitivity, score2, score3]
        return scores

    def boss_method(self, min_dist=4, max_dist=10, show_plot=True):
        """Run all methods above """
        # Load rosbag
        bag_path = bag_directory + self.bag_name
        lidar, pose = self.extract_data(bag_path)
        planes, ramp_stats, true_dist = self.run_the_algorithm(lidar, pose)
        df, ramp_stats = self.convert_to_df(planes, ramp_stats)
        df_stats = self.ground_truth_check(df, ramp_stats)
        if show_plot:
            self.some_plot(true_dist, df_stats)
        scores = self.calc_score(df_stats, true_dist, min_dist, max_dist)
        return scores

### Run algorithm and evaluation for each rosbag

In [ ]:
# Names of rosbags to evaluate
bag_names = [
    "u_c2s_half_odom_hdl.bag",
    "u_d2e_hdl.bag",
]

# Ground truth ramp region for each bag
xy_range = [
    [[20.3, 33], [-0.9, 2.8]],
    [[32.5, 44], [2, 5.5]],
]

scores = []
for i, bag_name in enumerate(bag_names):
    # Extract ramp region data
    x_range, y_range = xy_range[i]
    # Create instance of class
    gs = GetScore(bag_name, x_range, y_range)
    # Run algorithm + evaluation
    score = gs.boss_method()
    scores.append(score)